# 2. The Database: pgstac

pgstac is a PostgreSQL extension that enables STAC metadata management in a PostgreSQL database.
eoAPI is useful to many organizations because the other components are configured to work seamlessly with STAC metadata that is stored in your pgstac database.

**pypgstac** is a Python package for interacting with a pgstac database. You will learn  how to use pypgstac to perform the following operations on a pgstac database:
1. Generate STAC collection record
2. Add the record to the `collections` table with `Loader.load_collections`
3. Generate STAC item records
4. Add new records to the `items` table with `Loader.load_items`
5. Delete an item from the `items` table

For production deployments your STAC metadata generation and ingestion workflow will probably not take place in a notebook but the basic steps will be the same!

Fill in the input boxes below to get started with your own personal Sentinel-2 STAC collection! You will be using the `username` value to create a unique STAC collection for your home region that you will use in subsequent parts of the workshop. You will need to enter the database credentials here in order to post data to the database in this notebook.

In [23]:
import os

import ipywidgets as widgets
from IPython.display import display

username_input = widgets.Text(
    value="", placeholder="Enter your username", description="username:", disabled=False
)

lat_input = widgets.BoundedFloatText(
    value=38.9,
    min=-90,
    max=90,
    placeholder="enter the latitude of your hometown",
    description="latitude:",
    disabled=False,
)
lon_input = widgets.BoundedFloatText(
    value=-77.0,
    min=-180,
    max=180,
    placeholder="enter the longitude of your hometown",
    description="longitude:",
    disabled=False,
)

db_user_input = widgets.Text(
    value=os.getenv("POSTGRES_USER", "pgstac_user"),
    placeholder="Enter database username",
    description="db user:",
    disabled=False,
)
db_password_input = widgets.Text(
    value=os.getenv("POSTGRES_PASSWORD", ""),
    placeholder="Enter database password",
    description="db password:",
    disabled=False,
)
db_host_input = widgets.Text(
    value=os.getenv("POSTGRES_HOST", ""),
    placeholder="Enter database host address",
    description="db host:",
    disabled=False,
)
db_port_input = widgets.Text(
    value=os.getenv("POSTGRES_PORT", "5432"),
    placeholder="Enter database port",
    description="db port:",
    disabled=False,
)
db_dbname_input = widgets.Text(
    value=os.getenv("POSTGRES_DBNAME", "pgstac"),
    placeholder="Enter database dbname",
    description="db dbname:",
    disabled=False,
)

# Display the widgets
display(username_input)
display(lat_input)
display(lon_input)
display(db_user_input)
display(db_password_input)
display(db_host_input)
display(db_port_input)
display(db_dbname_input)

Text(value='', description='username:', placeholder='Enter your username')

BoundedFloatText(value=38.9, description='latitude:', max=90.0, min=-90.0)

BoundedFloatText(value=-77.0, description='longitude:', max=180.0, min=-180.0)

Text(value='pgstac_user', description='db user:', placeholder='Enter database username')

Text(value='', description='db password:', placeholder='Enter database password')

Text(value='', description='db host:', placeholder='Enter database host address')

Text(value='5432', description='db port:', placeholder='Enter database port')

Text(value='pgstac', description='db dbname:', placeholder='Enter database dbname')

## 2.1 Load a collection object

Start by creating a collection that will contain items within ~2 degrees of your provided location between January 1 2025 and April 18 2025.

In [29]:
from datetime import datetime

import pystac_client
from pystac import Collection, Extent, SpatialExtent, TemporalExtent
from pypgstac.db import PgstacDB
from pypgstac.load import Loader, Methods
from shapely.geometry import Point

stac_api_endpoint = os.getenv("STAC_API_ENDPOINT")

collection_id = f"{username_input.value}-sentinel-2-c1-l2a"
bbox = Point(lon_input.value, lat_input.value).buffer(2).bounds
temporal_extent = [datetime(2025, 1, 1), datetime(2025, 4, 18)]

my_collection = Collection(
    id=collection_id,
    description=f"{username_input.value}'s personal Sentinel-2 L2A collection",
    extent=Extent(
        spatial=SpatialExtent([[*bbox]]),
        temporal=TemporalExtent([temporal_extent]),
    ),
)
my_collection

<Collection id=hrodmn-sentinel-2-c1-l2a>

To connect to the `pgstac` database and upload data we need to generate the `dsn` and instantiate a `PgstacDB` and `Loader` pair.

In [30]:
db_creds = {
    "password": db_password_input.value,
    "dbname": "pgstac",
    "engine": "postgres",
    "port": 5432,
    "host": db_host_input.value,
    "username": db_user_input.value,
}
pgstac_dsn = "postgresql://{username}:{password}@{host}:{port}/{dbname}".format(
    **db_creds
)

db = PgstacDB(dsn=pgstac_dsn)
loader = Loader(db)

The `load_collections` method accepts an iterable of STAC collection dictionaries or a file path to a collection.json file or a .ndjson file with multiple collection records. You already have your `pystac.Collection` in memory in this session so you can just write it to a dictionary and upload it (in a list). 

The `upsert` method will add your collection to the `collections` table if it does not exist and update any fields with new values if the record does already exist.

In [31]:
loader.load_collections([my_collection.to_dict()], insert_mode=Methods.upsert)

Now if you run a query on the database you will find a record for your collection!

In [32]:
list(db.query(f"SELECT id from collections where id = '{my_collection.id}';"))

[('hrodmn-sentinel-2-c1-l2a',)]

## 2.2 Upload items

You already learned how to create STAC items from scratch in the [STAC metadata](./stac_metadata.ipynb) notebook so you get to take a shortcut for this one. Instead of creating STAC metadata from scratch you can just copy some from an existing STAC collection - this will be sufficient for our exercises during the workshop.

This code will find Sentinel-2 L2A items for the first few months of 2025 that intersect the bounding box of a 2 degree radius around the coordinate that you entered at the top of the notebook.

In [33]:
source_client = pystac_client.Client.open("https://earth-search.aws.element84.com/v1")

search = source_client.search(
    collections="sentinel-2-c1-l2a",
    bbox=bbox,
    datetime=temporal_extent,
    limit=100,  # pagination limit
)

items = search.item_collection()

print(len(items))

1205


You are going to upload these items to the items table in the `pgstac` database but to do so you need to make sure the items' collection ID matches an existing collection - set it to match the collection that you uploaded in the previous step.

In [34]:
# override the collection id to match your new collection
for item in items:
    item.set_collection(my_collection)

items[0]

<Item id=S2B_T15TXK_20250417T170051_L2A>

Now you can use the `load_items` method to upload the list of STAC item dictionaries to the items table in the `pgstac` database. The `insert_ignore` method will upload any items that do not exist in the table and skip records that already exist.

In [35]:
loader.load_items([item.to_dict() for item in items], insert_mode=Methods.insert_ignore)

Hooray you uploaded some items! Now run a query to verify that the records landed in the items table.

In [36]:
n_items = db.query_one(
    f"SELECT COUNT(*) FROM items where collection = '{my_collection.id}';"
)
print(f"there are {n_items} items")

there are 1205 items


Here is your first glimpse of the power of STAC metadata. You just uploaded these items to the database and now you can browse the data in a beautiful interface without doing any more work!

[Radiant Earth](https://radiant.earth/) built a tool called [STAC Browser](https://radiantearth.github.io/stac-browser/#/?.language=en) that is a human-readable interface to a STAC API. Your collection is immediately availble in the STAC API so you can browse it in STAC Browser. Take a tour around the STAC Browser page for your collection. 

In [37]:
from IPython.display import IFrame

IFrame(
    f"https://radiantearth.github.io/stac-browser/#/external/{stac_api_endpoint}/collections/{my_collection.id}",
    1200,
    800,
)

## 2.3 Delete an item

It is less common but you might want to delete a STAC record some day. There are not yet any `pypgstac` functions for doing this but it is relatively easy to do via PostgreSQL query.

Try deleting the last item in the list.

In [38]:
with db.connect() as conn:
    cur = conn.cursor()
    cur.execute(f"DELETE FROM items where id = '{items[-1].id}';")
    cur.close()
    conn.commit()

In [39]:
new_n_items = db.query_one(
    f"SELECT COUNT(*) FROM items where collection = '{my_collection.id}';"
)
print(f"now there are {new_n_items} items")

now there are 1204 items


In [40]:
# put it back :)
loader.load_items([items[-1].to_dict()], insert_mode=Methods.insert_ignore)

## 2.4 Search with pypgstac

It is not going to be the best option for searching a pgstac database in most cases (you should probably send a request to stac-fastapi-pgstac), but you can use pypgstac to search for items. You can provide a JSON request body like you would provide to a STAC API in the `query` arg of `PgstacDB.search`:

In [41]:
search_results = db.search(query={"collections": [my_collection.id], "limit": 1})

print(search_results)

{"type":"FeatureCollection","links":[{"rel":"root","href":".","type":"application/json"},{"rel":"self","href":"./search","type":"application/json"},{"rel":"next","href":"./search?token=next:hrodmn-sentinel-2-c1-l2a:S2B_T15TXK_20250417T170051_L2A","type":"application/geo+json","method":"GET"}],"features":[{"id":"S2B_T15TXK_20250417T170051_L2A","bbox":[-91.371536,44.13526,-90.332392,45.137034],"type":"Feature","links":[{"rel":"self","href":"https://earth-search.aws.element84.com/v1/collections/sentinel-2-c1-l2a/items/S2B_T15TXK_20250417T170051_L2A","type":"application/geo+json"},{"rel":"canonical","href":"s3://e84-earth-search-sentinel-data/sentinel-2-c1-l2a/15/T/XK/2025/4/S2B_T15TXK_20250417T170051_L2A/S2B_T15TXK_20250417T170051_L2A.json","type":"application/json"},{"rel":"via","href":"s3://sentinel-s2-l2a/tiles/15/T/XK/2025/4/17/0/metadata.xml","type":"application/xml","title":"Granule Metadata in Sinergize RODA Archive"},{"rel":"parent","href":"https://earth-search.aws.element84.com/v